# Tarea Unidad 3 - Aprendizaje Supervisado

Fecha de entrega: **Viernes 11 de Junio de 2021 a las 23:59**.

- Consultas por slack o al correo felipe dot quezada at uach dot cl
- 4 integrantes como maximo.

Integrantes:
- Ricardo Coronado
- Diego Sandoval


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

La tarea consiste en resolver 2 problemas utilizando los contenidos vistos hasta ahora. En la carpetas `datos/` se encuentran el dataset para el segundo problema. Para el primero problema, existe un link a un archivo compartido en google drive, que se encuentra mas abajo.

# 1: Clasificacion de Asteroides

**ALERTA! IMPACTO DE ASTEROIDE!** La nueva base espacial de Pluto esta constantemente bajo ataque de asteroides provenientes de un agujero blanco cercano. No son asteroides muy grandes, pero si han causado importantes daños en la infraestructura. Por esta razon, la Union Intergalactica Terrestre le ha pedido a usted, como experto en aprendizaje supervisado, hacerse cargo del problema. La base cuenta con equipamiento especial para detectar y pulverizar a los asteroides, sin embargo, como los recursos son limitados, se debe usar correctamente. Las características del asteroide se pueden calcular al instante con instrumentos especializados, solo hace falta alguna manera de poder utilizar esa información y predecir cuando son peligrosos y cuando no.

Para lograr la tarea, se ha puesto a su disposicion un listado de asteroides con sus caracteristicas y su etiqueta. El dataset esta compuesto por los siguientes campos:

- name: Nombre del asteroide
- a: Eje semi-mayor (grad)
- e: excentricidad (km)
- i: Inclinacion con respecto al plano eliptico x-y (grados)
- om: Longitud en el nodo ascendente (km)
- w: Valor de Perihelio (km)
- q: Distancia de Perihelio (km)
- ad: Distancia de Afelio (km)
- per_y: Periodo orbital (años)
- data_arc: Usado para cruzar info.
- condition_code: condicion del asteroide. De peor a mejor: 0, 1, 2, 3, 4, 5, 6, 7, D, E
- n_obs_used: numero de mediciones
- H: parametro de Magnitud Absoluta
- pha: VARIABLE OBJETIVO, pregunta, el asteroide presenta un peligro? N o Y
- diameter: Diametro (km)
- extent: extencion (km x km x km)
- albedo: porcentje de radiacion reflejada (Bq)
- rot_per: velocidad rotacion (m/s)
- GM: Masa global (m)
- BV: Volumen (m3)
- UB: Cantidad de Ultravioleta reflejada (Bq)
- IR: Cantidad de IR reflejados (Bq)
- G: Gravedad esperada (m/s2)
- moid: ID interno para cruzar datos
- class: Tipo de asteroide, 0, 1, 2, 3, 4, 5, 6, 7
- n: Rotacion (grados/dias)
- per: Periodo orbital (dias)
- ma: Anomalia media en grados.

**Objetivo Principal**
- Entrenar un Regresor Logistico capaz de predecir si el asteroide es peligroso o no.

El dataset se encuentra en: 

https://drive.google.com/file/d/1g_7HvzXQCtZ57X3inevTE4mQ49gZU6FW/view?usp=sharing


Para esto debe utilizar todos los contenidos vistos en la unidad. Esto incluye, pero no se limita a limpiar, preprocesar y segregar el dataset, entrenar y evaluar el modelo, etc... Cada decision que pueda ser considerada importante debe ser justificada, es decir, debe explicar por que no esta haciendo las cosas de otra manera. Recuerde, analize correctamente los datos, y sea consciente de lo que este haciendo su codigo para evitar errores *paranormales*. Finalmente debe terminar con un resumen de los principales desafios y resultados, comentando los resultados en funcion de las metricas vistas en clase. Para el Accuracy, comentar por que cree que obtuvo ese resultado.


## Respuesta
Se seleccionaron los siguientes parámetros para realizar la estimación de riesgo: condition_code, n_obs_used.  
Para la validación se realizó una comparación con: pha

En condition_code:   
Se reemplazó NaN por 9   
Se reemplazó D por 8   
Se reemplazó E por 9   

In [ ]:
# Descargue y localice el archivo Asteroides en carpeta datos
df = pd.read_csv("datos/Asteroides.csv", sep=';')
df['asteroide'] = np.arange(0,len(df),step=1)
df = df.drop(['name','a','e','i','om','w','q','ad','per_y','data_arc','H','albedo','rot_per','UB','IR','G','moid','class','n','per','ma','diameter','extent','GM','BV'], axis=1)
df = df[(df.pha != 'N')] # Dropea sólo los asteroides que no presenten un peligro claro
df['condition_code'] = df['condition_code'].replace(np.nan,9)
df['condition_code'] = df['condition_code'].replace('D',8)
df['condition_code'] = df['condition_code'].replace('E',9)
df = df[['asteroide','condition_code','n_obs_used','pha']]
df

In [ ]:
#OJO, asteroide es una variable identificadora, no forma parte del estudio

X = df.iloc[:, :-1].values 
y = df.iloc[:, 3].values


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# 2: Hallando la respuesta de la vida

Un grupo de arqueologos ha logrado encontrar la ultima pieza de una antigua tablilla sumeria con la respuesta de la vida, y necesitan de tu ayuda para descifrar sus secretos. Normalmente, las tablillan muestran una onda tal que al ser reproducidas en un altavoz, resuenan con otro artefacto sumerio a una cierta frecuencia. Esta frecuencia despues es traducida en una palabra utilizando un diccionario especial. Desafortunadamente, la ultima onda presenta perdida de informacion y ruido, debido al transcurso de los años. 

Hasta ahora solo han podido descifrar "La Respuesta de la Vida esta en ______".

Su mision es tomar la poca informacion que se tiene de la onda y tratar de descifrar la señal original.

**Objetivo Principal**
- Entrenar un Regresor Lineal para estimar la señal original.

Se espera que el error de reconstruccion sea el menor posible. Para esto, puede medir el error promedio de las predicciones de su modelo utilizando una metrica como el Error Medio Cuadratico.

$$\frac{1}{N}\sum_{i=0}^N(y_i - \hat y_i)^2$$

donde $y_i$ es la etiqueta, $\hat y_i$ es la prediccion de su modelo y $N$ el numero de observaciones. Se recomienda analizar la informacion que se tiene para hacer predicciones mas precisas.

**El entregable** sera este mismo notebook modificado con todo el procedimiento y los comentarios. Enviar al correo indicado al inicio del documento.
Se evaluara el procedimiento y la calidad de las justificaciones, ademas de los resultados obtenidos.

In [ ]:
X = pd.read_csv("datos/X.csv", sep=';')
y = pd.read_csv("datos/y.csv", sep=';')

In [ ]:
fig, ax = plt.subplots(figsize=(6,4), tight_layout=True)
ax.plot(X,y, 'bo')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)